# web-to-email

In [ ]:
url = "https://en.wikipedia.org"
email = "john@example.com"

In [2]:
import boto3

In [3]:
import os
os.environ['AWS_PROFILE'] = 'vbalasu_admin'

In [4]:
lam = boto3.client('lambda')

In [5]:
function_name = 'web-to-png-pdf-ConvertFunction-KUR1ON362oK7'

In [6]:
payload = bytes('{"queryStringParameters": {"url": "URL_GOES_HERE"}}'.replace('URL_GOES_HERE', url), 'utf-8')

In [7]:
response = lam.invoke(FunctionName=function_name, Payload=payload, LogType='Tail')

In [8]:
import json
html = json.loads(response['Payload'].read())['body']
#from IPython.display import HTML
#HTML(html)

In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [10]:
img = soup.find_all('img')[0]
img_base64 = img.get_attribute_list('src')[0].split(',')[1]

In [11]:
import base64
with open('/tmp/website.png', 'wb') as f:
    f.write(base64.b64decode(img_base64))

In [12]:
pdf = soup.find_all('iframe')[0]
pdf_base64 = pdf.get_attribute_list('src')[0].split(',')[1]

In [13]:
with open('/tmp/website.pdf', 'wb') as f:
    f.write(base64.b64decode(pdf_base64))

In [14]:
ses = boto3.client('ses')
import datetime
now = datetime.datetime.utcnow().isoformat()

In [15]:
subject = f'{url} as of {now}'

In [16]:
from email.message import EmailMessage
msg = EmailMessage()
msg['Subject'] = subject
msg['From'] = 'support@cloudmatica.com'
msg['To'] = email
with open('/tmp/website.png', 'rb') as f:
    file_data = f.read()
    file_name = f.name
msg.add_attachment(file_data, maintype='image', subtype='png', filename=file_name)
with open('/tmp/website.pdf', 'rb') as f:
    file_data = f.read()
    file_name = f.name
msg.add_attachment(file_data, maintype='application', subtype='pdf', filename=file_name)

In [17]:
ses.send_raw_email(RawMessage={'Data': msg.as_bytes()})

{'MessageId': '0100017d92bb17fd-847193ee-584a-4eb7-83f7-5d9fe2fd7060-000000',
 'ResponseMetadata': {'RequestId': '20c23722-eed0-4bf9-b669-265b68b9e0db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 07 Dec 2021 02:31:14 GMT',
   'content-type': 'text/xml',
   'content-length': '338',
   'connection': 'keep-alive',
   'x-amzn-requestid': '20c23722-eed0-4bf9-b669-265b68b9e0db'},
  'RetryAttempts': 0}}

In [56]:
email = 'john@example.com'
url = 'https://google.com'

In [10]:
from urllib.parse import quote_plus, unquote_plus

In [57]:
unquote_plus(quote_plus(email))

'john@example.com'

In [18]:
import boto3
dynamodb = boto3.resource('dynamodb')

In [29]:
dynamodb.create_table(TableName='web-to-email-config', AttributeDefinitions=[
            {
                'AttributeName': 'id',
                'AttributeType': 'S'
            }
],         KeySchema=[
            {
                'AttributeName': 'id',
                'KeyType': 'HASH'  # Partition key
            }
        ], BillingMode='PAY_PER_REQUEST'
)

ResourceInUseException: An error occurred (ResourceInUseException) when calling the CreateTable operation: Attempt to change a resource which is still in use: Table is being created: web-to-email-config

In [27]:
table = dynamodb.Table(name='web-to-email-config')

In [54]:
daily_reports = table.get_item(Key={'id': 'daily_reports'})['Item']['data']